In [75]:
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
import pandas as pd
from math import log
import openpyxl

In [71]:
corpers = pd.read_excel("/Users/seojeongrok/Desktop/코퍼스.xlsx")

In [72]:
corpers

,발화의 목적,발화 형식,다양한 발화 형식의 형태 변화
0,인사,반가워,안녕 안녕하세요 반갑다 반가워요 반갑습니다 좋은 아침 반갑
1,예약,으로 예약 할래,NaN
2,~ 예약,"바꿀, 거야",NaN
3,예약 수정,바꿔줘,NaN
4,예약 가능성,"별로야, 다른 건?",NaN
5,추가 변경 거부,아니,NaN
6,예약 확정,"좋아, 훌륭해, 마음에 들어, 완벽해",NaN
7,식당 장소 확인,주소,NaN
8,식당 연락처 확인,연락처,NaN
9,감사 인사,"고마워, 감사",NaN


In [73]:
corpers["다양한 발화 형식의 형태 변화"][0]

'안녕 안녕하세요 반갑다 반가워요 반갑습니다 좋은 아침 반갑'

In [76]:
Stopwords = Stopwords()
# kiwi 객체 생성
kiwi = Kiwi(num_workers=2, load_default_dict=True, integrate_allomorph=True, model_type='sbg')

In [86]:
docs = [
    "안녕 난 유방암에 걸린 서정록이야",
    "반가워 난 폐암에 걸린 서정록"
]

In [87]:
vocab = set()

for i in range(len(docs)):
    tokens = kiwi.tokenize(docs[i])
    for j in range(len(tokens)):
        if tokens[j].tag.startswith("E"):
            pass
        else:
            if tokens[j].form in corpers["다양한 발화 형식의 형태 변화"][0]:
                vocab.add(corpers["발화의 목적"][0])
            else:
                vocab.add(tokens[j].form)

setvocab = list(vocab)
print(vocab)

{'록', '유방암', '서정', '이', 'ᆫ', '나', '폐암', '에', '인사', '걸리'}


In [88]:


for i in range(len(docs)):
    doc_token = []
    tokens = kiwi.tokenize(docs[i])
    for j in range(len(tokens)):
        if tokens[j].tag.startswith("E"):
            pass
        elif tokens[j].tag.startswith("J"):
            pass
        else:
            if tokens[j].form in corpers["다양한 발화 형식의 형태 변화"][0]:
                doc_token.append((corpers["발화의 목적"][0], tokens[j].form))
            else:
                doc_token.append(tokens[j].form)
            
    print(doc_token)

[('인사', '안녕'), '나', '유방암', '걸리', '서정', '록', '이']
[('인사', '반갑'), '나', '폐암', '걸리', '서정', '록']


In [85]:

for i in range(len(docs)):
    tokens = kiwi.tokenize(docs[i])
    for j in range(len(tokens)):
        if tokens[j].tag.startswith("J"):
            pass
        elif tokens[j].tag.startswith("E"):
            pass
        elif tokens[j].tag.startswith("S"):
            pass
        else:
            vocab.add(tokens[j].form)

setvocab = list(vocab)
print(setvocab)

['록', '유방암', ',', '서정', '이', 'ᆫ', '반갑', '나', '안녕', '폐암', '에', '인사', '걸리']


In [25]:
# 총 문서의 수
N = len(docs)

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df+1))

def tfidf(t, d):
    return tf(t, d) * idf(t)

In [26]:
result = []


for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(setvocab)):
        t = setvocab[j]
        result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns=setvocab)

In [22]:
tf_

,유방암,의,1,이,요,?,이란,무엇,증상,원인,기,ᆫ가,은
0,1,0,0,1,1,1,1,1,0,0,0,0,0
1,1,1,0,0,1,1,0,1,0,1,0,0,1
2,1,0,1,0,0,1,0,0,1,0,1,0,1


In [21]:
result = []
for j in range(len(setvocab)):
    t = setvocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=setvocab, columns=["IDF"])

print(id)

<built-in function id>
